<a href="https://colab.research.google.com/github/JavierPachas/tf_examples/blob/main/keras_regression_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install seaborn

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

try :
  #tensorflow_version only exists in Colab
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
dataset_path = keras.utils.get_file("housing.data","https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data")

In [ ]:
dataset_path

In [ ]:
column_names = ['CRIM',"ZN","INDUS","CHAS","NOX","RM","DIS","RAD","TAX",'PTRATIO',"B","MEDV"]

In [ ]:
raw_dataset = pd.read_csv(dataset_path, names = column_names,
                          na_values = "?", comment = "\t",
                          sep =" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset.isna().sum()
#dataset = datset.dropna()

In [ ]:
train_dataset = dataset.sample(frac = 0.8, random_state = 0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[['CRIM','INDUS','TAX','MEDV']],
             diag_kind = 'kde')

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop('MEDV')
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('MEDV')
test_labels = test_dataset.pop('MEDV')

In [ ]:
def norm(x):
  return (x-train_stats['mean']) / train_stats['std']

In [ ]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
normed_train_data.tail()

In [ ]:
def build_model():
  model = keras.Sequential([
      layers.Dense(64, activation = 'relu',
                   input_shape = [len(train_dataset.keys())]),
      layers.Dense(64, activation = 'relu'),
      layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss = 'mse',
                optimizer = optimizer,
                metrics = ['mae','mse'])

  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

### Training a network

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end = '')

In [ ]:
EPOCHS = 10000

In [ ]:
history = model.fit(
    normed_train_data, train_labels,
    epochs = EPOCHS, validation_split = 0.2, verbose = 0,
    callbacks = [PrintDot()]
)

In [ ]:
#hist = pd.DataFrame(history.history)
#hist['epoch'] = history.epoch
#hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MEDV]')
  plt.plot(hist['epoch'], hist['mae'],
           label ='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Squared Error [$MEDV^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label = 'Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()

  plt.show()



In [ ]:
plot_history(history)